<a href="https://colab.research.google.com/github/GuiCoelhoDev/NLP_UnB_2022_1/blob/atividade3/Atividade3_160123046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enunciado

# Deve-se elaborar o Notebook Python descrevendo um processo de recuperação de textos visando retornar as notícias mais similares em relação à uma notícia de consulta. Utilize e compare as representações Bag-of-Words e TF-IDF, como também a métrica de similaridade Cosseno.



O que deve ser entregue:

O link de um repositório no GitHub contendo um notebook Python com a implementação do referido processo de recuperação de notícias. Finalmente, apresente para cada representação textual a validação do processo: a acurácia média de recuperação das 10 primeiras notícias recuperadas conforme as categorias (veja no corpus). Detalhando a seguir:

- Para cada notícia do corpus (considere como sendo a notícia de consulta) deve-se criar um ranking de similaridade em relação às demais notícias (excluindo-se a notícia de consulta). Deve-se verificar se a classe das top-10 notícias recuperadas na busca por similaridade pertencem a mesma classe da notícia de consulta. Se a i-ésima posição (dentre as top-10) são da mesma classe da notícia de consulta, contabilize o acerto. Faça isso para as primeiras 10 posições apenas!

       * Nesse caso, recomenda-se apresentar uma tabela com 10 posições, indicando a porcentagem de acerto (ou recuperação correta) das notícias buscadas.

       * Não se preocupar no momento com as notícias que estejam nas pastas de teste e treinamento (ainda não estudamos isso).

ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia\'s exporting\n  nations that the row could inflict far-reaching economic\n  damage, businessmen and officials said.\n      They told Reuter correspondents in Asian capitals a U.S.\n  Move against Japan might boost protectionist sentiment in the\n  U.S. And lead to curbs on American imports of their products.\n      But some exporters said that while the conflict would hurt\n  them in the long-run, in the short-term Tokyo\'s loss might be\n  their gain.\n      The U.S. Has said it will impose 300 mln dlrs of tariffs on\n  imports of Japanese electronics goods on April 17, in\n  retaliation for Japan\'s alleged failure to stick to a pact not\n  to sell semiconductors on world markets at below cost.\n      Unofficial Japanese estimates put the impact of the tariffs\n  at 10 billion dlrs and spokesmen for major electronics firms\n  said they would virtually halt exports of products hit by the\n  new taxes.\n      "We wouldn\'t be able to do business," said a spokesman for\n  

# Imports

In [18]:
import nltk
import pandas as pd
import math
nltk.download('reuters')
from nltk.corpus import reuters
import numpy as np
import cProfile
import re

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')


[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funções

In [19]:
#@title Pré Processamento
def preProcessing(texts):

    corpus = []
    for text in texts:
        text = text.lower()
        text_regexed = regexFiltering(text)
        text_no_sw = removeStopWords(text_regexed)
        text_stem_lem = LemAndStemm(text_no_sw)

        corpus.append(text_stem_lem)


    tokens = []
    for noticia in corpus:
        # noticia = ASIAN EXPORTERS FEAR DAMAGE FROM
        for word in noticia.split():

            tokens.append(word)

    tokens = sorted(set(tokens))

    vocab = tokens
    return corpus, vocab

In [20]:
#@ Lemmatization and Stemming
def LemAndStemm(text):
    lemmatizer = WordNetLemmatizer()
    stemmatizer = PorterStemmer()

    tokens = re.split(' ',text)

    lemas = [lemmatizer.lemmatize(t) for t in tokens]
    stemmas = [stemmatizer.stem(t) for t in lemas]
    text_stem_lem = ' '.join(stemmas)
    return text_stem_lem

In [21]:
#@ filtro regex
def regexFiltering(text):
    # text_regexed = re.sub(r'','',text)
    text_regexed = re.sub(r'&lt[^\s]+','', text)
    text_regexed = re.sub(r'[^a-zA-Z ]+','', text_regexed)

    return text_regexed

In [22]:
#@ removing stop words
def removeStopWords(text):
    stop_list = nltk.corpus.stopwords.words('english')
    tokens_no_sw = []
    for word in text.split():
        if word not in stop_list:
            tokens_no_sw.append(word)

    text_no_sw = ' '.join(tokens_no_sw)

    return text_no_sw

In [23]:
#@title Gerando Bag of Words de Contagem
def generateBowContagem(corpus):
    bow_contagem = {}
    for i,noticia in enumerate(corpus):
        bow_contagem['noticia {}'.format(i)] = {}
        for word in noticia.split():
            if word in bow_contagem['noticia {}'.format(i)]:
                bow_contagem['noticia {}'.format(i)][word] += 1
            else:
                bow_contagem['noticia {}'.format(i)][word] = 1

    df_bow_contagem = pd.DataFrame().from_records(bow_contagem).fillna(0).T.astype(int)
    return df_bow_contagem, bow_contagem

In [24]:
#@title Gerando Bag of Words Ponderado
def generateBowPonderado(corpus):
    bow_ponderado = {}
    N = {}
    # para cada noticia do corpus, fazer:
    for i,noticia in enumerate(corpus):
        # inicia dicionario para cada notícia
        bow_ponderado['noticia {}'.format(i)] = {}
        # N vai armazenar a quantidade de tokens de cada notícia
        N['noticia {}'.format(i)] = len(noticia.split())
        # para cada palavra de uma noticia
        for word in noticia.split():
            # se a palavra já está no dicionário da noticia, aumenta 1
            if word in bow_ponderado['noticia {}'.format(i)]:
                bow_ponderado['noticia {}'.format(i)][word] += 1
            else:
                bow_ponderado['noticia {}'.format(i)][word] = 1

    for noticia in bow_ponderado:
        for word in bow_ponderado[noticia]:
            bow_ponderado[noticia][word] /= N[noticia]


    df_bow_ponderado = pd.DataFrame().from_records(bow_ponderado).fillna(0).T.astype(float)

    return df_bow_ponderado, bow_ponderado

In [25]:
#@title Gerando TF-IDF
def generateTfidf(vocab,corpus, bow_contagem, bow_ponderado):
    tf = {} # frequencia de termos em todo o corpus
    idf = {}
    num = len(corpus)
    for word in vocab:
        tf[word] = 0
        for noticia in bow_contagem:
            if word in bow_contagem[noticia]:
                tf[word] += 1
    for word in tf:

        den = tf[word]
        idf[word] = math.log(num/den)

    bow_tfidf = {}
    for noticia in bow_ponderado:
        bow_tfidf[noticia] = {}
        for word in bow_ponderado[noticia]:
            a = bow_ponderado[noticia][word]
            b = idf[word]
            bow_tfidf[noticia][word] = a*b
    df_bow_tfidf = pd.DataFrame().from_records(bow_tfidf).fillna(0).T
    return df_bow_tfidf, bow_tfidf

In [26]:
#@title Similaridade Cosseno
def cossineSimilarity(vetor1, vetor2):
    prod_interno = 0

    for elemento1, elemento2 in zip(vetor1,vetor2):
        prod_interno += elemento1*elemento2

    norma_vetor1 = 0
    for x in vetor1:
        norma_vetor1 += x * x
    norma_vetor1 = norma_vetor1 ** 0.5

    norma_vetor2 = 0
    for x in vetor2:
        norma_vetor2 += x * x
    norma_vetor2 = norma_vetor2 ** 0.5

    return prod_interno / (norma_vetor1 * norma_vetor2)

In [27]:
#@title Gerando dataframe de similaridade
def generateDfSim(df_bow_tfidf):

    df_similaridades = pd.DataFrame([])
    df_bow_tfidf.reset_index(inplace=True, drop=True)
    for i in range(0, len(df_bow_tfidf)):
        print(f"Similaridade da noticia {i}")
        similaridades = []

        # preenchendo a linha 0 de similaridade
        vetor1 = df_bow_tfidf.iloc[i].values.flatten().tolist()
        for j in range(0, len(df_bow_tfidf)):
            vetor2 = df_bow_tfidf.iloc[j].values.flatten().tolist()
            cos_sim = cossineSimilarity(vetor1, vetor2)
            similaridades.append(cos_sim)


        df_similaridade = pd.DataFrame([similaridades])
        df_similaridades = pd.concat([df_similaridades, df_similaridade], ignore_index=True)

    np.fill_diagonal(df_similaridades.values, 0)
    return df_similaridades

In [28]:
#@title Gerando dataframe top10 
def  generateDfTop10(df_similaridades, df_reduced):

    indices_noticias_mais_similares_top10 = df_similaridades.apply(lambda s: s.abs().nlargest(10).index.tolist(), axis=1)
    df_top_10 = pd.DataFrame(0,index=np.arange(1), columns=np.arange(1,11))


    for indice_noticia_consulta, lista in enumerate(indices_noticias_mais_similares_top10):
    # para cada lista contendo as top 10 noticias mais similares
        # ['trade']
        noticia_consulta_categorias = df_reduced.iloc[indice_noticia_consulta]['categories']
        # [['ipi','earn'], ['bop'], ['tin']]
        lista_tuplas_noticia_categoria = []
        for indice_noticia_similar in lista:
        #
            tupla_noticia_categoria = (indice_noticia_similar, df_reduced.iloc[indice_noticia_similar]['categories'])
            lista_tuplas_noticia_categoria.append(tupla_noticia_categoria)
        ranking = 1
        for tupla in lista_tuplas_noticia_categoria:
            for categoria in noticia_consulta_categorias:
                if categoria in tupla[1]:
                    df_top_10[ranking] += 1
                    break
            ranking += 1

    return df_top_10

In [29]:
#@title  main
def main():
    cats = reuters.categories()

    fileids = reuters.fileids()

    categories = []
    text = []

    for file in fileids:
        categories.append(reuters.categories(file))
        text.append(reuters.raw(file))

    df = pd.DataFrame({'ids': fileids, 'categories': categories, 'text': text})

    df_reduced = df.sample(500)

    texts = df_reduced['text']

    corpus, vocab = preProcessing(texts)
    df_bow_contagem, bow_contagem = generateBowContagem(corpus)
    df_bow_ponderado, bow_ponderado = generateBowPonderado(corpus)
    df_bow_tfidf, bow_tfidf = generateTfidf(vocab,corpus,bow_contagem, bow_ponderado)
    df_similaridades = generateDfSim(df_bow_tfidf)

    df_top_10 = generateDfTop10(df_similaridades, df_reduced)
    pd.Series(vocab).to_csv("vocabulario.csv")
    print("\n")
    return df_top_10

# Execução

In [30]:
df_top_10 = main()
print("\nTabela de acurácia TOP-10\n")
df_top_10

Similaridade da noticia 0
Similaridade da noticia 1
Similaridade da noticia 2
Similaridade da noticia 3
Similaridade da noticia 4
Similaridade da noticia 5
Similaridade da noticia 6
Similaridade da noticia 7
Similaridade da noticia 8
Similaridade da noticia 9
Similaridade da noticia 10
Similaridade da noticia 11
Similaridade da noticia 12
Similaridade da noticia 13
Similaridade da noticia 14
Similaridade da noticia 15
Similaridade da noticia 16
Similaridade da noticia 17
Similaridade da noticia 18
Similaridade da noticia 19
Similaridade da noticia 20
Similaridade da noticia 21
Similaridade da noticia 22
Similaridade da noticia 23
Similaridade da noticia 24
Similaridade da noticia 25
Similaridade da noticia 26
Similaridade da noticia 27
Similaridade da noticia 28
Similaridade da noticia 29
Similaridade da noticia 30
Similaridade da noticia 31
Similaridade da noticia 32
Similaridade da noticia 33
Similaridade da noticia 34
Similaridade da noticia 35
Similaridade da noticia 36
Similaridad

,1,2,3,4,5,6,7,8,9,10
0,94,78,82,87,81,88,90,94,88,91


In [35]:
print("Tabela de TOP-10 Considerando 500 notícias\n")

df_top_10

Tabela de TOP-10 Considerando 500 notícias



,1,2,3,4,5,6,7,8,9,10
0,94,78,82,87,81,88,90,94,88,91
